In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
seed =1 

In [2]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=0)
train_labels=data_train["cancelled"]
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders',
                            'delivered_time','cancelled','order_id'],axis=1)
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train
X = train_examples
y = train_labels

In [6]:
def baseline_model():
    model = Sequential()
    model.add(Dense(6, input_dim=6, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=100, verbose=False)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator.fit(X, y)
prediction = estimator.predict(X)
prediction=np.where(prediction > 0.5, 1, 0)

accuracy_score(y, prediction)

2022-02-05 11:33:10.892635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:18.634646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:19.229373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:26.852909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:27.386898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:34.918739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:33:35.451849: I tensorflow/core/grappler/optimizers/cust

Results: -0.59 (0.15) MSE


2022-02-05 11:34:34.528397: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-05 11:34:42.777163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


0.9298311111111112

In [7]:
test=pd.read_csv("data/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','undelivered_orders',
                            'order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)
ans=estimator.predict(test)
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)
data.head()

2022-02-05 11:38:05.737838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [ ]:
data.to_csv("data/model1_23.csv",index=False)